In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/Shareddrives/CS263-Project/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Part 1: Generate specificity and commitment labels using fine-tuned ClimateBERT

In [2]:
!pip install transformers==4.28.0

In [3]:
pages = {
    "Broadridge_2021": [7, 14],
    "IPG_2021": [9, 23],
    "Juniper_2021": [22, 35],
    "Norton_2021": [19, 21],
    "GlobalPayments_2021": [36, 43],
    "Microchip_2021": [20, 29],
    "NetApp_2021": [29, 35],
    "Paychex_2021": [50, 62],
    "Paycom_2021": [17, 18],
    "Qorvo_2022": [25, 33],
    "WesternUnion_2021": [54, 58]
}

In [4]:
test_data = {}
file_names = []
max_seq_len = 1024
test_data_dicts = {}
averaged_labels = {}

path = 'data/reports'
for file in os.listdir(path):
  if os.path.isfile(path + '/' + file):
    name = file.split('.')[0]
    if name in pages.keys():
      test_data[name] = []
      file_names.append(name)
      test_data_dicts[name] = {}
      averaged_labels[name] = {}
      print(name)
      with open(path + '/' + file, "r") as text:
        raw = text.read()
        i = 0
        data_id = 0
        while i < len(raw):
          temp = raw[i:i+max_seq_len]
          test_data_dicts[name][data_id] = {
              "text": None,
              "sentiment": None,
              "tcfd": None,
              "specificity": None,
              "commitment": None,
              "detection": None
          }
          offset = 0
          # End segments at end of sentence if possible, otherwise at a new line
          for j in range(len(temp)-1, -1, -1):
            if temp[j] == '.':
              test_data[name].append(temp[:j+1])
              test_data_dicts[name][data_id]["text"] = temp[:j+1]
              offset = j + 1
              break
          if offset == 0:
            for j in range(len(temp)-1, -1, -1):
              if temp[j] == '\n':
                test_data[name].append(temp[:j+1])
                test_data_dicts[name][data_id]["text"] = temp[:j+1]
                offset = j + 1
                break
          i += offset
          data_id += 1

# print(test_data["NVIDIA_2021"][-1])

Broadridge_2021
Microchip_2021
IPG_2021
GlobalPayments_2021
NetApp_2021
Paychex_2021
Norton_2021
Paycom_2021
Qorvo_2022
Juniper_2021
WesternUnion_2021


In [5]:
file_names

['Broadridge_2021',
 'Microchip_2021',
 'IPG_2021',
 'GlobalPayments_2021',
 'NetApp_2021',
 'Paychex_2021',
 'Norton_2021',
 'Paycom_2021',
 'Qorvo_2022',
 'Juniper_2021',
 'WesternUnion_2021']

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

dataset_name = "climatebert/climate_specificity"
spec_model = "climatebert/distilroberta-base-climate-specificity"
comm_model = "climatebert/distilroberta-base-climate-commitment"

model = AutoModelForSequenceClassification.from_pretrained(spec_model)
tokenizer = AutoTokenizer.from_pretrained(spec_model)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

spec_outputs = {}
for file in file_names:
  spec_outputs[file] = {'label': 'non', 'score': 0}
  for i, out in enumerate(tqdm(pipe(test_data[file], padding=True, truncation=False))):
    if out['label'] == 'non':
      spec_outputs[file]['score'] += 1 - out['score']
      test_data_dicts[file][i]["specificity"] = 0
    else:
      spec_outputs[file]['score'] += out['score']
      test_data_dicts[file][i]["specificity"] = 1
  spec_outputs[file]['score'] /= len(test_data[file])
  if spec_outputs[file]['score'] >= 0.5:
    spec_outputs[file]['label'] = 'spec'

print(spec_outputs)

model = AutoModelForSequenceClassification.from_pretrained(comm_model)
tokenizer = AutoTokenizer.from_pretrained(comm_model)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

comm_outputs = {}
for file in file_names:
  comm_outputs[file] = {'label': 'no', 'score': 0}
  for i, out in enumerate(tqdm(pipe(test_data[file], padding=True, truncation=False))):
    if out['label'] == 'no':
      comm_outputs[file]['score'] += 1 - out['score']
      test_data_dicts[file][i]["commitment"] = 0
    else:
      comm_outputs[file]['score'] += out['score']
      test_data_dicts[file][i]["commitment"] = 1
  comm_outputs[file]['score'] /= len(test_data[file])
  if comm_outputs[file]['score'] >= 0.5:
    comm_outputs[file]['label'] = 'yes'

print(comm_outputs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

{'Broadridge_2021': {'label': 'yes', 'score': 0.867384155591329}, 'Microchip_2021': {'label': 'yes', 'score': 0.6011647946304746}, 'IPG_2021': {'label': 'yes', 'score': 0.8792689392964045}, 'GlobalPayments_2021': {'label': 'yes', 'score': 0.9020489893461529}, 'NetApp_2021': {'label': 'yes', 'score': 0.9214488376270641}, 'Paychex_2021': {'label': 'yes', 'score': 0.5881720532973608}, 'Norton_2021': {'label': 'yes', 'score': 0.9022312879562377}, 'Paycom_2021': {'label': 'yes', 'score': 0.6422370970249176}, 'Qorvo_2022': {'label': 'yes', 'score': 0.7621577789908961}, 'Juniper_2021': {'label': 'yes', 'score': 0.8828040605936295}, 'WesternUnion_2021': {'label': 'yes', 'score': 0.7090608676274618}}


In [7]:
for file in file_names:
  averaged_labels[file]["specificity"] = 1 if spec_outputs[file]['label'] == 'spec' else 0
  averaged_labels[file]["commitment"] = 1 if comm_outputs[file]['label'] == 'yes' else 0

### Part 2: Generate New Sentiment
This is where we'll run the text through the sentiment model to receive a binary label separate from ClimateBERT. The code in this section is adapted from: https://colab.research.google.com/github/chrsiebert/sentiment-roberta-large-english/blob/main/sentiment_roberta_prediction_example.ipynb#scrollTo=0wC0q6Bxp3or.

In [8]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [9]:
# Load tokenizer and model, create trainer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [10]:
# Create list of texts (can be imported from .csv, .xls etc.)
def get_text(data_dict):
  pred_texts = []
  data_order = []
  for id, vals in data_dict.items():
    data_order.append(id)
    pred_texts.append(data_dict[id]['text'])

  zipped = list(zip(data_order, pred_texts))
  sorted_zipped = sorted(zipped, key = lambda x: x[0])
  res = [[i for i, j in sorted_zipped],
       [j for i, j in sorted_zipped]]
  return res[0], res[1]

In [11]:
# map the labels to binary
def binarize_labels(labels_list):
  labels = []
  for label in labels_list:
    if label == "NEGATIVE":
      labels.append(0)
    elif label == "POSITIVE":
      labels.append(1)
    else:
      print("ERROR")
  return labels

In [12]:
# add the sentiment to the data dictionary
def add_sentiment_to_dict(id_list, label_list, data_dict):
  print(id_list)
  assert id_list == sorted(id_list)
  for index, id in enumerate(id_list):
    data_dict[id]["binary_sentiment"] = label_list[index]
  return data_dict

In [13]:
for file in file_names:
  data_order_test, test_text = get_text(test_data_dicts[file])

  assert data_order_test == sorted(data_order_test)
  tokenized_test = tokenizer(test_text,truncation=True,padding=True)
  test_dataset = SimpleDataset(tokenized_test)
  test_predictions = trainer.predict(test_dataset)

  test_preds = test_predictions.predictions.argmax(-1)
  test_labels = pd.Series(test_preds).map(model.config.id2label)
  test_scores = (np.exp(test_predictions[0])/np.exp(test_predictions[0]).sum(-1,keepdims=True)).max(1)

  test_labels = binarize_labels(test_labels)

  print(test_labels)

  averaged_labels[file]["binary_sentiment"] = round(sum(test_labels)/len(test_labels))

  test_data_dicts[file] = add_sentiment_to_dict(data_order_test, test_labels, test_data_dicts[file])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


...

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[1, 1]
[0, 1]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [14]:
!pip install prettyprint

  Preparing metadata (setup.py) ... done
  Created wheel for prettyprint: filename=prettyprint-0.1.5-py3-none-any.whl size=3026 sha256=7d467f0296d1500a7bd083ba238a68acaf79512dcd220238ed49b18626d41928
  Stored in directory: /root/.cache/pip/wheels/b2/d0/51/477413885481c635ab7c6400f96f47b8a0971bbc1241ff9c9f
Successfully built prettyprint


In [15]:
from pprint import PrettyPrinter
# print(test_data_dicts["NVIDIA_2021"][0])
pp = PrettyPrinter()
pp.pprint(averaged_labels)

{'Broadridge_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'GlobalPayments_2021': {'binary_sentiment': 1,
                         'commitment': 1,
                         'specificity': 1},
 'IPG_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'Juniper_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'Microchip_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'NetApp_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'Norton_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'Paychex_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'Paycom_2021': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'Qorvo_2022': {'binary_sentiment': 1, 'commitment': 1, 'specificity': 1},
 'WesternUnion_2021': {'binary_sentiment': 1,
                       'commitment': 1,
                       'specificity': 0}}


### Deflection

In [19]:
deflection_language = [
    "alleged",
    "purported",
    "ostensibly",
    "arguably",
    "presumably",
    "putatively",
    "seemingly",
    "reportedly",
    "apparent",
    "so-called",
    "allegedly",
    "potentially",
    "possibly",
    "putative",
    "suggested",
    "circumstantially",
    "put forth",
    "by all accounts",
    "inferred",
    "conceivably",
    "inconclusively",
    "tentatively",
    "speculatively",
    "hypothetically",
    "in all likelihood",
    "plausibly",
    "implied",
    "presumptively",
    "theoretically",
    "presumed",
    "conditionally",
    "preliminarily",
    "provisionally",
    "notoriously",
    "put forward",
    "understood",
    "it is said",
    "subject to interpretation",
    "perceived",
    "in a sense",
    "to a degree",
    "assumed",
    "putatively",
    "vaguely",
    "broadly",
    "loosely",
    "dubiously",
    "evidently",
    "inferred",
    "outwardly",
    "reputedly",
    "tacitly",
    "unofficially",
    "putatively",
    "ostensible",
    "presumptuous",
    "unconfirmed",
    "not aware",
    "unsure"
]

In [20]:
for file in file_names:
  test_deflection_label = []
  for id in test_data_dicts[file].keys():
    text = test_data_dicts[file][id]['text'].lower()
    # test_deflection_label.append(sum([text.count(term)for term in deflection_language]))
    def_label = max([1 if term in text else 0 for term in deflection_language])
    test_deflection_label.append(def_label)
    test_data_dicts[file][id]["deflection"] = def_label
  print(sum(test_deflection_label))
  # test["deflection"] = test_deflection_label

0
0
0
0
0
2
0
0
0
0
0


In [33]:
count = 0
for file_dict in test_data_dicts.values():
  count += len(file_dict)
print("%d test data samples processed" % (count))

199 test data samples processed


In [38]:
all_file_names = []
path = 'data/reports'
for file in os.listdir(path):
  if os.path.isfile(path + '/' + file):
    name = file.split('.')[0]
    all_file_names.append(name)

count = 0
for file in all_file_names:
  test = pd.read_csv("./data/"+file+"-data.csv")
  count += test.shape[0]

print("%d total test samples from %d reports" % (count, len(all_file_names)))

532 total test samples from 17 reports


### Part 3: Save New Data

In [28]:
import csv

# given the dictionary and the new filename, write the information appropriately
def write_labels(data_dict, filename):
 with open(filename, mode='w') as data_file:
  csv_writer = csv.writer(data_file, delimiter=',')
  csv_writer.writerow(['number', 'text', 'sentiment', 'binary_sentiment', 'tcfd', 'climate_specificity', 'commitment', 'detection', 'deflection', 'greenwashing_risk'])

  for id, vals in data_dict.items():
    csv_writer.writerow([str(id), data_dict[id]['text'], str(data_dict[id]['sentiment']), str(data_dict[id]['binary_sentiment']), str(data_dict[id]['tcfd']), str(data_dict[id]['specificity']), str(data_dict[id]['commitment']), str(data_dict[id]['detection']), str(data_dict[id]['deflection'])])

In [29]:
for file in file_names:
  write_labels(test_data_dicts[file], "data/"+file+"-data.csv")

### Part 4: Annotate for Greenwashing
Recall our current naive rule:
$$x = (2 - sentiment) + specificity + commitment + deflection$$

$$ z = \begin{cases}
  0  & x > 2 \\
  1 & x \leq 2
\end{cases}$$

In [ ]:
def read_csv_file(filename):
  data_dict = {}

  with open(filename) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader: # row is a list at this point
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
      else:
        data_id = int(row[1])
        text = row[2]
        sentiment = row[3]
        binary_sentiment = int(row[4])
        tcfd = row[5]
        specificity = int(row[6])
        commitment = int(row[7])
        detection = row[8]
        greenwashing_risk = row[9]
        deflection = int(row[10])
        data_dict[data_id] = {
            "text": text,
            "sentiment": sentiment,
            "binary_sentiment": binary_sentiment,
            "tcfd": tcfd,
            "specificity": specificity,
            "commitment": commitment,
            "detection": detection,
            "greenwashing_risk": greenwashing_risk,
            "deflection": deflection
        }
      line_count += 1

    return data_dict

In [ ]:
import math
# given the dictionary, we're going to implement the naive annotation and add it to each data examples
def annotate_groundtruth(data_dict):
  # arr = []
  # scores = []
  for data_id, val_dict in data_dict.items():
    sentiment = data_dict[data_id]["binary_sentiment"]
    specificity = data_dict[data_id]["specificity"]
    commitment = data_dict[data_id]["commitment"]
    deflection = data_dict[data_id]["deflection"]

    raw_score = 0.71 * sentiment + 0.14 * specificity - 0.86 * commitment - 0.71 * deflection
    probability = math.exp(raw_score) / (1 + math.exp(raw_score))
    label = int(probability >= 0.67)

    # arr.append(x)
    # scores.append(score)

    data_dict[data_id]["greenwashing_risk"] = label
  # print('arr', arr)
  # print('score', scores)
  return data_dict

In [ ]:
def write_annotation(data_dict, filename):
 with open(filename, mode='w') as data_file:
  csv_writer = csv.writer(data_file, delimiter=',')
  csv_writer.writerow(['number', 'text', 'sentiment', 'binary_sentiment', 'tcfd', 'climate_specificity', 'commitment', 'detection', 'greenwashing_risk'])

  for id, vals in data_dict.items():
    csv_writer.writerow([str(id), data_dict[id]['text'], str(data_dict[id]['sentiment']), str(data_dict[id]['binary_sentiment']), str(data_dict[id]['tcfd']), str(data_dict[id]['specificity']), str(data_dict[id]['commitment']), str(data_dict[id]['detection']), str(data_dict[id]['greenwashing_risk'])])

In [ ]:
for file in file_names:
  test_dict = read_csv_file("data/"+file+"_deflection.csv")
  test_dict_truth = annotate_groundtruth(test_dict)
  write_annotation(test_dict_truth, "data/"+file+"-final.csv")

Column names are , number, text, sentiment, binary_sentiment, tcfd, climate_specificity, commitment, detection, greenwashing_risk, deflection
Column names are , number, text, sentiment, binary_sentiment, tcfd, climate_specificity, commitment, detection, greenwashing_risk, deflection
Column names are , number, text, sentiment, binary_sentiment, tcfd, climate_specificity, commitment, detection, greenwashing_risk, deflection
Column names are , number, text, sentiment, binary_sentiment, tcfd, climate_specificity, commitment, detection, greenwashing_risk, deflection
Column names are , number, text, sentiment, binary_sentiment, tcfd, climate_specificity, commitment, detection, greenwashing_risk, deflection
Column names are , number, text, sentiment, binary_sentiment, tcfd, climate_specificity, commitment, detection, greenwashing_risk, deflection
